In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ECommerceAnalysis').getOrCreate()
df = spark.read.csv("/FileStore/sample_ecommerce_data1.csv", header=True, inferSchema=True)
df.show()

+--------------+----------+----------------+-------------------+-------------+-------------+-------------------+-----------+-----------------+--------------+
|Transaction ID|Product ID|Product Category|Product Subcategory|Product Price|Quantity Sold|   Transaction Date|Customer ID|Customer Location|Payment Method|
+--------------+----------+----------------+-------------------+-------------+-------------+-------------------+-----------+-----------------+--------------+
|             1|       151|        Clothing|             Mobile|        11.14|           11|2023-02-12 00:00:00|       1017|          Florida|        PayPal|
|             2|       192|     Electronics|            Fiction|       237.04|           10|2023-01-21 00:00:00|       1016|         New York|    Debit Card|
|             3|       114|           Books|            Fiction|       154.13|           19|2023-11-02 00:00:00|       1037|         New York|    Debit Card|
|             4|       171|       Groceries|        

In [0]:
from pyspark.sql.functions import col, when

df_cleaned = df.dropna()
df_cleaned = df.filter(col('Product ID') > 100)
df_cleaned.show()
df_renamed = df_cleaned.withColumnRenamed("Product Category", "Product Category name")
df_renamed.show()
df_transformed = df_renamed.withColumn('new_column', col('Quantity Sold') * 2)
df_transformed.show()


+--------------+----------+----------------+-------------------+-------------+-------------+-------------------+-----------+-----------------+--------------+
|Transaction ID|Product ID|Product Category|Product Subcategory|Product Price|Quantity Sold|   Transaction Date|Customer ID|Customer Location|Payment Method|
+--------------+----------+----------------+-------------------+-------------+-------------+-------------------+-----------+-----------------+--------------+
|             1|       151|        Clothing|             Mobile|        11.14|           11|2023-02-12 00:00:00|       1017|          Florida|        PayPal|
|             2|       192|     Electronics|            Fiction|       237.04|           10|2023-01-21 00:00:00|       1016|         New York|    Debit Card|
|             3|       114|           Books|            Fiction|       154.13|           19|2023-11-02 00:00:00|       1037|         New York|    Debit Card|
|             4|       171|       Groceries|        

In [0]:
from pyspark.sql.functions import sum, count, avg


total_sales_per_category = df_transformed.groupBy("Product ID").agg(
    sum("Product Price").alias("total_sales")
)

total_sales_per_category.show()




+----------+------------------+
|Product ID|       total_sales|
+----------+------------------+
|       148|           2134.65|
|       137|3266.0299999999997|
|       133|            2127.6|
|       108|           2003.65|
|       155|           2032.56|
|       193|2037.5700000000002|
|       101|3451.3199999999993|
|       126|2290.9700000000003|
|       115|            3026.0|
|       183|2401.4300000000003|
|       192|           3014.21|
|       159|           2493.29|
|       103|3142.6800000000003|
|       128|           1566.72|
|       122|3746.3599999999997|
|       157|           3766.04|
|       190|           1152.82|
|       111|           2909.67|
|       177|           1299.81|
|       140| 651.3100000000001|
+----------+------------------+
only showing top 20 rows



In [0]:
df_transformed.write.csv("/FileStore/sample_ecommerce_data_transformed", mode="overwrite")
df_transformed.show()

+--------------+----------+---------------------+-------------------+-------------+-------------+-------------------+-----------+-----------------+--------------+----------+
|Transaction ID|Product ID|Product Category name|Product Subcategory|Product Price|Quantity Sold|   Transaction Date|Customer ID|Customer Location|Payment Method|new_column|
+--------------+----------+---------------------+-------------------+-------------+-------------+-------------------+-----------+-----------------+--------------+----------+
|             1|       151|             Clothing|             Mobile|        11.14|           11|2023-02-12 00:00:00|       1017|          Florida|        PayPal|        22|
|             2|       192|          Electronics|            Fiction|       237.04|           10|2023-01-21 00:00:00|       1016|         New York|    Debit Card|        20|
|             3|       114|                Books|            Fiction|       154.13|           19|2023-11-02 00:00:00|       1037| 

In [0]:
total_sales_per_category.write.parquet("/FileStore/sample_ecommerce_data", mode="overwrite")
total_sales_per_category.show()

+----------+------------------+
|Product ID|       total_sales|
+----------+------------------+
|       148|           2134.65|
|       137|3266.0299999999997|
|       133|            2127.6|
|       108|           2003.65|
|       155|           2032.56|
|       193|2037.5700000000002|
|       101|3451.3199999999993|
|       126|2290.9700000000003|
|       115|            3026.0|
|       183|2401.4300000000003|
|       192|           3014.21|
|       159|           2493.29|
|       103|3142.6800000000003|
|       128|           1566.72|
|       122|3746.3599999999997|
|       157|           3766.04|
|       190|           1152.82|
|       111|           2909.67|
|       177|           1299.81|
|       140| 651.3100000000001|
+----------+------------------+
only showing top 20 rows

